In [ ]:
## IMPORTS, VARIABLES & FUNCTIONS ##
import requests
from requests.auth import HTTPBasicAuth
import json
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine

var_table = 'Backlog'
var_ifExists = 'append' #overwrite current db? (fail, replace, or append)

# RESTful API endpoint
url = 'https://foo.atlassian.net/rest/api/3/'

# Personal access token 
token = 'foo'
userID = 'foo@foo.com'

# Auth
auth = HTTPBasicAuth(userID, token)

# Define JQL query 
JQL = 'project = "UE" AND issueType = "story" AND status = "backlog"'
#JQL = 'Key = "UE-2334"'

# Set up request headers with the token
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

# Make a GET request with headers
response = requests.get(f'{url}/search?jql={JQL}', params={'maxResults' : 1000, 'startAt' : 101}, headers=headers, auth=auth)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    print(json.dumps(data, indent=4))

    # Initialize lists to store data
    keys = []
    issue_types = []
    summaries = []
    reporters = []
    created_dates = []
    updated_dates = []
    statuses = []
    labels = []
    processes = []
    methods = []
    sprints = []

    #function to return a single value from an input that is part of a list
    def getListValue (
        field,
        field_key
    ):

        field = issue.get('fields', {}).get(field_key, '')
        if isinstance(field, list) and field:
            field_value = field[0].get('value', '')
            return(field_value)
        else:
            return('')

    # Iterate over each issue in the response
    for issue in data.get('issues', []):
        # Extract key, summary, and reporter for each issue
        key = issue.get('key', '')
        type = issue.get('fields', {}).get('issuetype', {}).get('name', '')
        summary = issue.get('fields', {}).get('summary', '')
        reporter = issue.get('fields', {}).get('reporter', {}).get('displayName', '')
        created = issue.get('fields', {}).get('created', '')
        updated = issue.get('fields', {}).get('statuscategorychangedate', '')
        status = issue.get('fields', {}).get('status', {}).get('name', '')
        label = issue.get('fields', {}).get('labels', '')
        process = getListValue(field='process', field_key='customfield_10707')
        method = getListValue(field='process', field_key='customfield_10708')
        sprint = issue.get('fields', {}).get('sprint', '')

        # Append data to lists
        keys.append(key)
        issue_types.append(type)
        summaries.append(summary)
        reporters.append(reporter)
        created_dates.append(created)
        updated_dates.append(updated)
        statuses.append(status) 
        labels.append(label)
        processes.append(process)
        methods.append(method)
        sprints.append(sprint)

    # Create DataFrame from the lists
    df = pd.DataFrame({'Key': keys, 'Type': issue_types, 'Summary': summaries, 'Reporter': reporters, 'Created': created_dates, 'Updated': updated_dates, 'Status': statuses, 'Labels': labels, 'Process': processes, 'Method': methods, 'Sprint': sprints})
    
    # Convert 'Created' and 'Updated' columns to datetime and remove time
    df['Created'] = pd.to_datetime(df['Created'], yearfirst=True, utc=True).dt.date
    df['Updated'] = pd.to_datetime(df['Updated'], yearfirst=True, utc=True).dt.date

    # Reorder the columns
    df = df[['Key', 'Type', 'Summary', 'Reporter', 'Created', 'Updated', 'Status', 'Sprint', 'Labels', 'Process', 'Method']]

    # Write the DF to a SQL DB
    df.to_sql(name=var_table, con=db_connection, if_exists=var_ifExists, index=False) 

    print(df.head())
else:
    print('Error:', response.status_code)
